<a href="https://colab.research.google.com/github/RoshaniPawar16/AAI/blob/main/Task1/Deep_Q_Learning_Task_1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def evaluate_agent(policy_net, episodes=1000, max_steps=500):
    """Evaluate the performance of the trained agent."""
    success_count = 0
    total_rewards = 0
    total_adjusted_rewards = 0
    total_steps = 0

    for episode in range(episodes):
        current_state, _ = env.reset()
        state = one_hot_encode(current_state, nspace)
        state = torch.tensor(np.array([state]), device=device)
        episode_rewards = 0
        episode_adjusted_rewards = 0
        steps = 0

        for step in range(max_steps):
            with torch.no_grad():
                action = policy_net(state).max(1)[1].view(1, 1)

            next_state, reward, done, _, _ = env.step(action.item())
            episode_rewards += reward

            # Adjusted reward calculation
            distance_to_goal = manhattan_distance(current_state)
            next_distance_to_goal = manhattan_distance(next_state)
            adjusted_reward = reward
            if done and reward == 1:
                adjusted_reward += 100  # Goal reached
            elif done and reward == 0:
                adjusted_reward -= 100  # Fell into a hole
            elif next_distance_to_goal < distance_to_goal:
                adjusted_reward += 2
            else:
                adjusted_reward -= 0.1 * next_distance_to_goal
            episode_adjusted_rewards += adjusted_reward

            current_state = next_state
            next_state = one_hot_encode(next_state, nspace)
            state = torch.tensor(np.array([next_state]), device=device)
            steps += 1

            if done:
                if reward == 1:
                    success_count += 1
                break

        total_rewards += episode_rewards
        total_adjusted_rewards += episode_adjusted_rewards
        total_steps += steps

    avg_success_rate = (success_count / episodes) * 100
    avg_reward = total_rewards / episodes
    avg_adjusted_reward = total_adjusted_rewards / episodes
    avg_steps = total_steps / episodes

    print(f"Evaluation Results over {episodes} episodes:")
    print(f"Success Rate: {avg_success_rate:.2f}%")
    print(f"Average Reward: {avg_reward:.2f}")
    print(f"Average Adjusted Reward: {avg_adjusted_reward:.2f}")
    print(f"Average Steps Taken: {avg_steps:.2f}")

    return avg_success_rate, avg_reward, avg_adjusted_reward, avg_steps


# Evaluate the agent
evaluate_agent(policy_net)


Evaluation Results over 1000 episodes:
Success Rate: 0.00%
Average Reward: 0.00
Average Adjusted Reward: -109.71
Average Steps Taken: 132.41


(0.0, 0.0, -109.70519999999996, 132.414)